In [10]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *

## INDATA
First, read indata from a specified excel file and sheet.

In [22]:
excel_path = "/Users/AlexanderLarsson/documents/VSCode/test_indata.xlsx"
indata_sheet = "test_case_1"
indataDF = pd.read_excel(excel_path,indata_sheet)
outdataDF = indataDF.copy()
indataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev
0,1,RDC,Johannesburg,10,40,1,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.1
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.2
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.3
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.4
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.5
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.6
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.7
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.8
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.9


In [23]:
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy()
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
sigma_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand type").to_numpy()
Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))

In [24]:

rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)

outdataDF.loc[outdataDF["Type"] == "RDC","Demand type"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean"] = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size/L_wh

outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Lead time demand mean,Lead time demand stdev
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,10.0,2.052691,100.0,20.526907
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,NaN,NaN
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,NaN,NaN
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,NaN,NaN
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.400000,NaN,NaN
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.500000,NaN,NaN
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.600000,NaN,NaN
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.700000,NaN,NaN
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.800000,NaN,NaN
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.900000,NaN,NaN


RDC reorder-point optimization

In [25]:
h_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Holding cost").to_numpy()
fill_rate_target_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Target item fill rate").to_numpy()
p_dealer_arr = fill_rate_target_arr*h_dealer_arr/(np.ones_like(fill_rate_target_arr)-fill_rate_target_arr)
l_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Transport time").to_numpy()
mu_wh = mu_L/L_wh * Q_subbatch_size

beta_list = []
for h,Q,p,l,my,sigma in zip(h_dealer_arr,Q_dealer_arr,p_dealer_arr,l_dealer_arr,mu_dealer_arr,sigma_dealer_arr):
    beta_list.append(induced_backorder_cost_opt(h,Q,p,l,my,sigma))

beta_arr = np.array(beta_list)

beta_rdc = weighting_backorder_cost(mu_dealer_arr,mu_wh,beta_arr)
outdataDF.loc[outdataDF["Type"] == "RDC", "Beta"] = beta_rdc
outdataDF.loc[outdataDF["Type"] == "Dealer", "Beta"] = beta_arr
print(f"Optimal weighted induced backorder cost at the warehouse is: {beta_rdc}, betas are: {beta_arr}")



Optimal weighted induced backorder cost at the warehouse is: 0.9864807894306118, betas are: [0.36044571 0.73766048 0.98969763 1.53249848 0.80103968 0.9947581
 1.3079709  0.8621658  1.02414387 1.25442723]


In [26]:
h_rdc = float(indataDF.get(indataDF["Type"] == "RDC").get("Holding cost"))
Q_0 = int(indataDF.get(indataDF["Type"] == "RDC").get("Q"))

R_0 = warehouse_optimization(Q_subbatch_size,Q_0,rdc_f_u_probability_array,h_rdc,beta_rdc)
outdataDF.loc[outdataDF["Type"] == "RDC","R optimal"] = R_0
outdataDF


Starting optimizing, R = 0, c = 169.10155994988622, c+1 = 176.56939868439073
Doing downwards optimizing, R = -1, c = 162.2782878027405, c+1 = 169.10155994988622
Doing downwards optimizing, R = -2, c = 156.09958315943103, c+1 = 162.2782878027405
Doing downwards optimizing, R = -3, c = 150.5654460199578, c+1 = 156.09958315943103
Doing downwards optimizing, R = -4, c = 145.67587638432076, c+1 = 150.5654460199578
Doing downwards optimizing, R = -5, c = 141.43087425251997, c+1 = 145.67587638432076
Doing downwards optimizing, R = -6, c = 137.83043962455542, c+1 = 141.43087425251997
Doing downwards optimizing, R = -7, c = 134.87457250042706, c+1 = 137.83043962455542
Doing downwards optimizing, R = -8, c = 132.56327288013497, c+1 = 134.87457250042706
Doing downwards optimizing, R = -9, c = 130.89654076367913, c+1 = 132.56327288013497
Doing downwards optimizing, R = -10, c = 129.8743761510595, c+1 = 130.89654076367913
Doing downwards optimizing, R = -11, c = 129.49677904227607, c+1 = 129.874376

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Lead time demand mean,Lead time demand stdev,Beta,R optimal
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,10.0,2.052691,100.0,20.526907,0.986481,-11.0
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,NaN,NaN,0.360446,NaN
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,NaN,NaN,0.737660,NaN
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,NaN,NaN,0.989698,NaN
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.400000,NaN,NaN,1.532498,NaN
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.500000,NaN,NaN,0.801040,NaN
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.600000,NaN,NaN,0.994758,NaN
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.700000,NaN,NaN,1.307971,NaN
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.800000,NaN,NaN,0.862166,NaN
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.900000,NaN,NaN,1.024144,NaN


In [27]:
W = waiting_time(negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)
outdataDF.loc[outdataDF["Type"]== "Dealer", "Waiting time"] = W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand mean").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] =outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand stdev").to_numpy()
outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Lead time demand mean,Lead time demand stdev,Beta,R optimal,Waiting time,Lead time
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,10.0,2.052691,100.000000,20.526907,0.986481,-11.0,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,16.551767,18.206944,0.360446,NaN,6.551767,16.551767
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,9.551767,11.462120,0.737660,NaN,6.551767,9.551767
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,8.551767,11.117297,0.989698,NaN,6.551767,8.551767
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.400000,7.551767,10.572474,1.532498,NaN,6.551767,7.551767
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.500000,10.551767,15.827651,0.801040,NaN,6.551767,10.551767
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.600000,9.551767,15.282827,0.994758,NaN,6.551767,9.551767
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.700000,8.551767,14.538004,1.307971,NaN,6.551767,8.551767
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.800000,11.551767,20.793181,0.862166,NaN,6.551767,11.551767
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.900000,10.551767,20.048357,1.024144,NaN,6.551767,10.551767


In [28]:
opt_dealer_list = []
L_dealer_arr = outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time")
for Q,L_est,fill_rate_target,demand_type,mu,sigma in zip(Q_dealer_arr,L_dealer_arr,fill_rate_target_arr,demand_type_arr, mu_dealer_arr,sigma_dealer_arr):
    print(demand_type, mu, sigma)
    opt_dealer_list.append(dealer_R_optimization(Q,L_est,fill_rate_target,demand_type,mu,demand_variance = math.pow(sigma,2)))

R_opt_dealer_list,fill_rate_dealer_list,exp_stock_on_hand_list = [],[],[]
for tup in opt_dealer_list:
    R_opt_dealer_list.append(tup[0])
    fill_rate_dealer_list.append(tup[2])
    exp_stock_on_hand_list.append(tup[3])

outdataDF.loc[outdataDF["Type"] == "Dealer", "R optimal"] = R_opt_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Realized item fill rate"] = fill_rate_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected stock on hand"] = exp_stock_on_hand_list

NBD 1.0 1.1
NBD 1.0 1.2
NBD 1.0 1.3
NBD 1.0 1.4
NBD 1.0 1.5
NBD 1.0 1.6
NBD 1.0 1.7
NBD 1.0 1.8
NBD 1.0 1.9
NBD 1.0 2.0


In [30]:
outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Lead time demand mean,Lead time demand stdev,Beta,R optimal,Waiting time,Lead time,Realized item fill rate,Expected stock on hand
0,1,RDC,Johannesburg,10,40,1,NaN,Normal,10.0,2.052691,100.000000,20.526907,0.986481,-11.0,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,1.100000,16.551767,18.206944,0.360446,21.0,6.551767,16.551767,0.955630,10.029772
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,1.200000,9.551767,11.462120,0.737660,13.0,6.551767,9.551767,0.953703,9.019667
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,1.300000,8.551767,11.117297,0.989698,13.0,6.551767,8.551767,0.963426,10.004021
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,1.400000,7.551767,10.572474,1.532498,12.0,6.551767,7.551767,0.958141,10.014490
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,1.500000,10.551767,15.827651,0.801040,17.0,6.551767,10.551767,0.959366,12.029387
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,1.600000,9.551767,15.282827,0.994758,16.0,6.551767,9.551767,0.953731,12.044088
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,1.700000,8.551767,14.538004,1.307971,16.0,6.551767,8.551767,0.960329,13.029373
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,1.800000,11.551767,20.793181,0.862166,21.0,6.551767,11.551767,0.957756,15.055685
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,1.900000,10.551767,20.048357,1.024144,20.0,6.551767,10.551767,0.952867,15.071396


In [11]:
W

0.0

In [12]:
rdc_f_u_probability_array

array([nan])